sklearnに付属しているcross_validation_score関数を用いずに、自分で定めたkFoldから交差分割検証を行なう。  
あくまで、validationの値を確認するためだということを忘れない。パラメータチューニングの効率的なやり方がないか調べた方がいい。

In [ ]:
from sklean.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import gc
import xgboost as xgb


# 分割の数を指定。基本は5で十分
NFOLDS = 5
# 層化交差分割検証を基本的には用いる。時系列データの場合は時間を基準にしてvalidationを行なう
kf = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=123)

# テストデータの予測値と、validationの値を格納
y_preds = np.zeros(X_test.shape[0])
y_oof = np.zeros(X_train.shape[0])
score = 0
  
for fold, (tr_idx, val_idx) in enumerate(kf.split(X_train, y_train)):
# モデルの作成。
    clf = xgb.XGBClassifier(
        n_estimators=500,
        max_depth=9,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        gamma = 0.2,
        alpha = 4,
        missing = -1,
        tree_method='gpu_hist'
    )
# 訓練データをtrain,validaiton用に分ける
    X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
    y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
# 学習
    clf.fit(X_tr, y_tr)
# validationの予測
    y_pred_val = clf.predict_proba(X_vl)[:,1]
# validaitonの予測値を格納
    y_oof[val_idx] = y_pred_val
# 問題ごとの評価指標でvalidationの値を確認
    print("FOLD: ",fold,' AUC {}'.format(roc_auc_score(y_vl, y_pred_train)))
# validaitonの平均値を格納
    score += roc_auc_score(y_vl, y_pred_train) / NFOLDS
# 各validationごとのモデルで求めたテストデータの予測値の平均を格納
    y_preds+= clf.predict_proba(X_test)[:,1] / NFOLDS
# ガベコレ
    del X_tr, X_vl, y_tr, y_vl
    gc.collect()
    
    
print("\nMEAN AUC = {}".format(score))
print("OOF AUC = {}".format(roc_auc_score(y_train, y_oof)))